## Getting resources

We use [xp5k](https://github.com/pmorillon/xp5k) to manage the interaction with the Grid resources.

We create a new ```XP``` and define a new job.

Note that ```resources``` are the same as those given in a ```oarsub``` command line.

In [1]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
    :resources => ["nodes=3, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["allow_classic_ssh"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 3})],
    :command   => "sleep 86400"
    })

## Submission

```submit``` is asynchronous, ```wait_for_jobs``` will wait for all the resources to be ready.

In [2]:
myxp.submit
myxp.wait_for_jobs

#<Proc:0x00000003f87318@/var/lib/gems/2.1.0/bundler/gems/xp5k-c34c75cd7400/lib/xp5k/xp.rb:118>

## Run command

```xp5k/rake``` include some helpers to run commands on the nodes of your reservation.

In [3]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'msimonin'} do 
    cmd = ['uptime -s']
end

Connected to sagittaire-55.lyon.grid5000.fr...
Connected to sagittaire-74.lyon.grid5000.fr...
Connected to sagittaire-9.lyon.grid5000.fr...
[command][sagittaire-55.lyon.grid5000.fr] uptime -s
[command][sagittaire-74.lyon.grid5000.fr] uptime -s
[command][sagittaire-9.lyon.grid5000.fr] uptime -s
[stdout][sagittaire-55.lyon.grid5000.fr] 2016-01-16 03:36:19

[stdout][sagittaire-74.lyon.grid5000.fr] 2016-01-16 23:14:26

[stdout][sagittaire-9.lyon.grid5000.fr] 2016-01-16 23:39:09

---- sagittaire-55.lyon.grid5000.fr ------------------------------------------------------------------------------------------------------------------------------------------------------------------------
["2016-01-16 03:36:19"]
---- sagittaire-74.lyon.grid5000.fr ------------------------------------------------------------------------------------------------------------------------------------------------------------------------
["2016-01-16 23:14:26"]
---- sagittaire-9.lyon.grid5000.fr ---------------------------

{"sagittaire-55.lyon.grid5000.fr"=>["2016-01-16 03:36:19"], "sagittaire-74.lyon.grid5000.fr"=>["2016-01-16 23:14:26"], "sagittaire-9.lyon.grid5000.fr"=>["2016-01-16 23:39:09"]}

## Draw the bar plot

The output of the above command is a map :

 * each key is a server name 
 * a value is the outputs of the commands run on this server
 
e.g : ``` 'sagittaire-1.lyon.grid5000.fr' => ['out1', ... , 'outn'] ```

The following code compute the time since last reboot in minute.

In [4]:
require 'nyaplot'
require 'time'

now = Time.now.to_i
plot = Nyaplot::Plot.new
df = Nyaplot::DataFrame.new({
    host: h.keys
           .map{|x| x.split('.').first},
    uptime: h.values
             .map{|x| x.first}
    .map{|x| ((now - Time.parse(x).to_f)/60).round(2)}
    })
    

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

host,uptime
sagittaire-55,1160.4
sagittaire-74,-17.72
sagittaire-9,-42.43


In [5]:
plot=Nyaplot::Plot.new
plot.add_with_df(df, :bar, :host, :uptime)
plot.configure do
    x_label('')
    y_label('Uptime (min)')
    rotate_x_label(-90)
end


#<Nyaplot::Plot:0x00000002412ab0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000002412880 @properties={:type=>:bar, :options=>{:x=>:host, :y=>:uptime}, :data=>"aab35a35-4ce7-4c9e-a448-a375de1a3460"}, @xrange=["sagittaire-55", "sagittaire-74", "sagittaire-9"], @yrange=[-42.43, 1160.4]>], :options=>{:x_label=>"", :y_label=>"Uptime (min)", :rotate_x_label=>-90, :width=>700, :xrange=>["sagittaire-55", "sagittaire-74", "sagittaire-9"], :yrange=>[-42.43, 1160.4]}}>

In [6]:
myxp.clean

[".xp_cache"]